In [4]:
from pydantic import BaseModel, Field

from qdrant_client import QdrantClient
from qdrant_client.models import Prefetch, Filter, FieldCondition, MatchText, FusionQuery, Document


from langsmith import traceable, get_current_run_tree

from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.types import Send, Command

from langchain_core.messages import AIMessage, ToolMessage

from jinja2 import Template
from typing import Literal, Dict, Any, Annotated, List, Optional, Sequence
from IPython.display import Image, display
from operator import add
from openai import OpenAI

import openai

import random
import ast
import inspect
import instructor
import json
from dataclasses import dataclass

/home/arminas_work/projects/AI-Engineering_Bootcamp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Output models


In [5]:
@dataclass
class RetrievalOutput:
    retrieved_context_id: str
    retrieved_context: str
    retrieved_context_rating: float
    similarity_score: float

In [6]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [7]:
def get_embedding(text:str, model:str ="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=[text],
        model=model,
    )
    return response.data[0].embedding

In [11]:
def retrieve_data(
        query:str, 
        qdrant_client: QdrantClient, 
        k:int=5
    )-> list[RetrievalOutput]:

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-01-hybrid-search",
        prefetch=[
            Prefetch(
                query=query_embedding,
                using="text-embedding-3-small",
                limit=20
            ),
            Prefetch(
                query=Document(
                    text=query,
                    model="qdrant/bm25"
                ),
                using="bm25",
                limit=20
            )
        ],
        query=FusionQuery(fusion="rrf"),
        limit=k,
    )

    retrieval_outputs = []

    for result in results.points:
        retrieval_output= RetrievalOutput(
            retrieved_context_id= result.payload["parent_asin"],
            retrieved_context= result.payload["description"],
            retrieved_context_rating = result.payload["average_rating"],
            similarity_score = result.score
        )
        retrieval_outputs.append(retrieval_output)



    return retrieval_outputs

In [9]:
query = "Can I get a tablet?"

In [12]:
answer = retrieve_data(query, qdrant_client, k=10)

In [13]:
answer

[RetrievalOutput(retrieved_context_id='B0B8NVNQKX', retrieved_context='COOPERS 7 inch Kids Tablet Android 11 Tablet for Kids, 2GB RAM + 32GB ROM Toddler Tablet PC for Children, IPS Touch Screen, Dual Camera, Dual Speaker, WiFi Computer Tablet, Light Blue✿【Good Kids tablet】This 7 inch tablet for kids with silm body and lightweight, it is easy to hold by children. Also the special design can protect the tablet well when dropping. ✿【Parental Control】Toddlers Tablet with parent mode can add or block apps. Set screen time limits. This tablet come with iwawa app. kids can get access to fun and educational games and videos. ✿【Powerful Tablets】Equipmented with quad core CPU, Android 11.0 System, 32GB Big Storage, 1024*600 IPS Screen offer a clear view. Runs Kinds of apps and game for kids smoothly. ✿【Long Lasting】Tablet for kids built with large capacity battery. For mixed use up to 6 hours. You can enjoy happy parent-child time with your kids. ✿【Best Gift】This high performance Children tablet

##Multi-intent Questions